# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
import pandas as pd
df = pd.read_csv('ToothGrowth.csv')
df.head()
#df.describe()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


In [2]:
df.describe()

,len,dose
count,60.000000,60.000000
mean,18.813333,1.166667
std,7.649315,0.628872
min,4.200000,0.500000
25%,13.075000,0.500000
50%,19.250000,1.000000
75%,25.275000,2.000000
max,33.900000,2.000000


In [3]:
df['supp'].unique()

array(['VC', 'OJ'], dtype=object)

In [4]:
df['dose'].unique()

array([0.5, 1. , 2. ])

In [5]:
len(df['len'].unique())

43

## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [6]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [7]:
formula = 'len ~ C(supp) + C(dose)' 
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=1)
print(table)

            df       sum_sq      mean_sq          F        PR(>F)
C(supp)    1.0   205.350000   205.350000  14.016638  4.292793e-04
C(dose)    2.0  2426.434333  1213.217167  82.810935  1.871163e-17
Residual  56.0   820.425000    14.650446        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [8]:
##while both seem to make an impact, dose is a higher factor than supp on tooth length

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [14]:
samp1 = df[df['supp']=='OJ']['len']
samp2 = df[df['supp']=='VC']['len']



Now run a t-test between these two groups and print the associated two-sided p-value: 

In [20]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
import scipy.stats as stats 
stats.ttest_ind(samp1, samp2, equal_var=False) #returns t stat and pvalue



Ttest_indResult(statistic=1.91526826869527, pvalue=0.06063450788093387)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [22]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
formula = 'len ~ C(supp)' 
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=1)
print(table)
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

            df       sum_sq     mean_sq         F    PR(>F)
C(supp)    1.0   205.350000  205.350000  3.668253  0.060393
Residual  58.0  3246.859333   55.980333       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [23]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [26]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
from itertools import combinations

groups = [group[0] for group in df.groupby(['supp', 'dose'])['len']]
combos = combinations(groups, 2)
for combo in combos:
    supp1 = combo[0][0]
    dose1 = combo[0][1]
    supp2 = combo[1][0]
    dose2 = combo[1][1]
    sample1 = df[(df.supp == supp1) & (df.dose == dose1)]['len']
    sample2 = df[(df.supp == supp2) & (df.dose == dose2)]['len']
    p = stats.ttest_ind(samp1, samp2, equal_var=False)[1]
    print(combo, p)
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

(('OJ', 0.5), ('OJ', 1.0)) 0.06063450788093387
(('OJ', 0.5), ('OJ', 2.0)) 0.06063450788093387
(('OJ', 0.5), ('VC', 0.5)) 0.06063450788093387
(('OJ', 0.5), ('VC', 1.0)) 0.06063450788093387
(('OJ', 0.5), ('VC', 2.0)) 0.06063450788093387
(('OJ', 1.0), ('OJ', 2.0)) 0.06063450788093387
(('OJ', 1.0), ('VC', 0.5)) 0.06063450788093387
(('OJ', 1.0), ('VC', 1.0)) 0.06063450788093387
(('OJ', 1.0), ('VC', 2.0)) 0.06063450788093387
(('OJ', 2.0), ('VC', 0.5)) 0.06063450788093387
(('OJ', 2.0), ('VC', 1.0)) 0.06063450788093387
(('OJ', 2.0), ('VC', 2.0)) 0.06063450788093387
(('VC', 0.5), ('VC', 1.0)) 0.06063450788093387
(('VC', 0.5), ('VC', 2.0)) 0.06063450788093387
(('VC', 1.0), ('VC', 2.0)) 0.06063450788093387


## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.